In [2]:
import pandas as pd
import numpy as np
import sqlite3
import re

In [3]:
def segment_sql_commands(sql_file):
    commands = []
    current_command = ""
    in_comment_block = False

    with open(sql_file, 'r') as file:
        try:
            for line in file:
                line = line.strip()

                if not line or line.startswith('--'):
                    continue

                if line.startswith('/*') and line.endswith('*/;'):
                    continue
                elif line.startswith('/*'):
                    in_comment_block = True
                    continue
                elif line.endswith('*/;'):
                    in_comment_block = False
                    continue
                
                if in_comment_block:
                    continue

                current_command += line + ' '

                if line.endswith(';'):
                    current_command = current_command.replace(' unsigned','')
                    current_command = current_command.replace(' AUTO_INCREMENT','')
                    commands.append(current_command.strip())
                    current_command = ""
        except:
            print("Error")

    return commands

# sql_file = "libgen_compact.sql"
sql_file = "fiction.sql"
segmented_commands = segment_sql_commands(sql_file)
comms = segmented_commands[:]

Error


In [5]:
for i in range(3,1000):
    with open("statements_fiction/statement"+ str(i) + ".txt",'w') as file:
        file.write(comms[i])

### The next step was to reduce the statement to a more manageable chunk. But I think this won't be necessary anymore so move to the next step.

In [5]:
def extract_rows(sql_insert):
    start_idx = sql_insert.find('(')
    end_idx = sql_insert.rfind(')')
    if start_idx == -1 or end_idx == -1:
        raise ValueError("Invalid SQL insert statement")

    rows = sql_insert[start_idx + 1:end_idx]
    return rows.split('),(')

def create_new_insert(original_insert, n):
    rows = extract_rows(original_insert)
    new_rows = rows[:n]
    new_insert = original_insert.split('(')[0] + '(' + '),('.join(new_rows) + ');'
    return new_insert

def main(input_file, output_file, n):
    with open(input_file, 'r') as f:
        sql_insert = f.read()

    new_insert = create_new_insert(sql_insert, n)

    with open(output_file, 'w') as f:
        f.write(new_insert)


In [ ]:
main("statement.txt", "reduced_statement.txt", 2000)

### Now skip the next code block too. We'll be using "sql_to_csv_NEW" function.

In [108]:
def sql_to_csv(statement_file,out_file):
    with open(statement_file,'r') as file:
        insert = file.readline()
        m1 = re.findall('`(.*?)`', insert)
        columns = m1[1:]
        print(columns,len(columns))
        s=r'(.*?),'*46
        # print(r'\('+s+r'(.*?)\)')
        rows = re.findall(r'\('+s+r'(.*?)\)', insert)

        with open(out_file,'w') as file2:
            pass
        with open(out_file,'a') as file2:
            for row in rows:
                file2.write(str(row[0])+ " " + row[1] + " " + row[5] + "\n")

In [109]:
sql_to_csv("sql.txt","titles_authors.txt")

['ID', 'Title', 'VolumeInfo', 'Series', 'Periodical', 'Author', 'Year', 'Edition', 'Publisher', 'City', 'Pages', 'PagesInFile', 'Language', 'Topic', 'Library', 'Issue', 'Identifier', 'ISSN', 'ASIN', 'UDC', 'LBC', 'DDC', 'LCC', 'Doi', 'Googlebookid', 'OpenLibraryID', 'Commentary', 'DPI', 'Color', 'Cleaned', 'Orientation', 'Paginated', 'Scanned', 'Bookmarked', 'Searchable', 'Filesize', 'Extension', 'MD5', 'Generic', 'Visible', 'Locator', 'Local', 'TimeAdded', 'TimeLastModified', 'Coverurl', 'Tags', 'IdentifierWODash'] 47


In [17]:
# for k in range(3000):
#     search = "("+str(start+k)+","
#     search2 = "("+str(start+k+1)+","
#     search3 = "("+str(start+k+4)+","
#     index = insert.find(search)
#     index2 = insert.find(search2)
#     index3 = insert.find(search3)
#     # print(index)
#     if index==-1 and index2==-1 and index3==-1:
#         last_index = start+k-1
#         break
#     elif index==-1 and index2!=-1:
#         continue
#     elif index==-1 and index2==-1 and index3!=-1:
#         k = k+3
#         continue

# for k in range(3000):
#     for i in range(11):
#         search = "(" + str(start + k + i) + ","
#         index = insert.find(search)
#         if index != -1:
#             break
    
#     if index==-1:
#         last_index=start+k-1
#         break
#     else:
#         k=k+i
#         continue

Important fields for me: ID (0), Title (1), Series 3, Periodical 4, Author 5, Publisher 8, City 9, Language 12, Topic 13, Library 14, Commentary 26, Coverurl 44, Tags 45

Indian name classifier: https://github.com/vsant/indian-name-classifier

So we just classify the names? Or is there something more interesting we can do. If there's some sort of textual information, like a summary, we can think of a sort of language model to understand if the context is indian or not. Depending on the data we have we can think of training it or not. Training is always better if the data is available.

In [4]:
def is_integer(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

In [5]:
def sql_to_csv_NEW(statement_file,out_file,start):
    with open(out_file,'w') as file:
        pass
    with open(statement_file,'r') as file:
        with open(out_file,'a') as file2:
            insert = file.readline()

            # print("Searching from ",start," to +3000")

            assert insert.find("("+str(start)+",")!=-1

            found_last = False
            last_index = None

            for i in range(len(insert)-1,len(insert)-3000,-1):
                # print(i)
                if(insert[i]=='('):
                    string = insert[i:i+11]
                    # print(string)
                    comma_index = string.find(",")
                    # print(comma_index,string[:comma_index])
                    if comma_index==-1:
                        continue
                    elif not is_integer(string[1:comma_index]):
                        continue
                    else:
                        last_index = int(string[1:comma_index])
                        break

            end = last_index
            
            for i in range(start,end+1):
                search = "("+str(i)+","
                try:
                    start_index = insert.index(search)
                except:
                    continue
                count_quotes = 0
                #title
                third = None
                fourth = None
                #author
                fifth = None
                sixth = None
                #series
                seventh = None
                eighth = None
                #language
                eleventh = None
                twelfth = None

                for j in range(start_index, len(insert)):
                    if insert[j] == "'":
                        count_quotes += 1
                        if count_quotes == 3:
                            third = j
                        elif count_quotes == 4:
                            fourth = j   
                        elif count_quotes == 5:
                            fifth = j
                        elif count_quotes == 6:
                            sixth = j
                        elif count_quotes==7:
                            seventh=j
                        elif count_quotes==8:
                            eighth=j
                        elif count_quotes==11:
                            eleventh=j
                        elif count_quotes==12:
                            twelfth=j
                            break
                if third is not None and fourth is not None and fifth is not None and sixth is not None and\
                     seventh is not None and eighth is not None and eleventh is not None and twelfth is not None:
                    file2.write(str(i) + "," + insert[third+1:fourth].replace(",","")+","+insert[fifth+1:sixth].replace(",","") + ","+insert[seventh+1:eighth].replace(",","") + ","+insert[eleventh+1:twelfth].replace(",","") + "\n")
            
            return end

In [6]:
# Finished filling up till file 67 (ID 114675) so need to fill after that. There was an issue.
end=0
for i in range(3,1000):
    end = sql_to_csv_NEW("statements_fiction/statement"+ str(i) + ".txt","csvs_fiction/csv"+ str(i) + ".txt",end+1)
    if(i%10==0):
        print(end,end='\n')
    else:
        print(".",end="")

.......28667
.........64594
.........99734
.........131011
.........168152
.........202876
.........237539
.........272392
.........306894
.........341170
.........375613
.........407835
.........435038
.........463659
.........491762
.........522133
.........552519
.........582332
.........611113
.........639968
.........670039
.........700457
.........730285
.........760153
.........787376
.........813762
.........843766
.........875480
.........905937
.........933558
.........961511
.........990420
.........1017566
.........1044951
.........1073651
.........1103071
.........1132267
.........1160012
.........1187394
.........1215199
.........1242655
.........1270351
.........1298656
.........1327314
.........1356326
.........1384880
.........1413619
.........1442123
.........1470731
.........1498009
.........1526232
.........1560137
.........1594475
.........1628151
.........1654810
.........1681302
.........1708012
.........1734676
.........1761319
.........1787822
.........1814451


AssertionError: 

I noticed that this is around the point where my assumptions go false again. Because all the IDs are now not in order but are randomly jumbled up, so I'll have to search the entire insert string to find each occurrence of an ID to find the largest ID. Might not be worth it, huh. 

But also btw the algorithm is inefficient - because I don't need to incrementally find each ID. To get the maximum ID, all I need to do is check the last ID on the list. Start from the back, and the first regex matching "(N," where N is some integer - find that N and take that as the largest number. Then search for all integers in the range you've got and if you don't find a particular index  ignore that.